In [1]:
# 首先导入所需第三方库
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from tqdm import tqdm
import os

In [2]:

# 获取文件路径函数
def get_files(dir_path):
    # args：dir_path，目标文件夹路径
    file_list = []
    for filepath, dirnames, filenames in os.walk(dir_path):
        # os.walk 函数将递归遍历指定文件夹
        for filename in filenames:
            # 通过后缀名判断文件类型是否满足要求
            if filename.endswith(".txt"):
                # 如果满足要求，将其绝对路径加入到结果列表
                file_list.append(os.path.join(filepath, filename))
            # elif filename.endswith(".txt"):
            #     file_list.append(os.path.join(filepath, filename))
    return file_list


In [3]:

# 加载文件函数
def get_text(dir_path):
    # args：dir_path，目标文件夹路径
    # 首先调用上文定义的函数得到目标文件路径列表
    file_lst = get_files(dir_path)
    # docs 存放加载之后的纯文本对象
    docs = []
    # 遍历所有目标文件
    for one_file in tqdm(file_lst):
        file_type = one_file.split('.')[-1]
        # if file_type == 'md':
        #     loader = UnstructuredMarkdownLoader(one_file)
        if file_type == 'txt':
            loader = UnstructuredFileLoader(one_file)
        else:
            # 如果是不符合条件的文件，直接跳过
            continue
        docs.extend(loader.load())
    return docs

In [8]:
# 目标文件夹
tar_dir = [
    # "/root/autodl-tmp/qwen",
    # "/root/autodl-tmp/Qwen",
    "./ops/zedx2txt"
]

# 加载目标文件
docs = []
for dir_path in tar_dir:
    docs.extend(get_text(dir_path))
docs

100%|██████████| 54306/54306 [11:55<00:00, 75.92it/s] 


[Document(page_content='缩略语\n\n1xCSFB\n\n1x Circuit Switched Fallback1x电路交换回落\n\n1xRTT\n\n1x Radio Transmission Technology1x无线传输技术\n\n2G\n\nThe 2nd Generation Mobile Communications第二代移动通信\n\n3G\n\nThe 3rd Generation Mobile Communications第三代移动通信\n\n3GPP\n\n3rd Generation Partnership Project第三代合作伙伴计划\n\n4A\n\nAccount, Authorization, Authentication, Audit认证、账号、授权、审计\n\n5G\n\nGUTI\n\n5G Globally Unique Temporary Identity5G全球唯一临时标识\n\n5GC\n\n5G Core Network5G核心网\n\n5GS\n\n5G System5G系统\n\n5QI\n\n5G QoS Indicator5G QoS指示\n\nA\n\nSMF\n\nAnchor\n\nSMF锚点SMF\n\nAAA\n\nAnswer\n\nAuth\n\nAnswer应答鉴权响应\n\nAAR\n\nAnswer\n\nAuth\n\nRequest应答鉴权请求\n\nACK\n\nAcknowledgement应答\n\nACL\n\nAccess Control List访问控制列表\n\nACR\n\nAccounting Request计费请求\n\nADD\n\nAutomatic Device Detection自动设备检测\n\nAES\n\nAdvanced Encryption Standard高级加密标准\n\nAF\n\nApplication Function应用功能\n\nAssured Forwarding确保转发\n\nAIA\n\nAuthentication\n\nInformation\n\nAnswer鉴权信息检索响应\n\nAIR\n\nAuthentication\n\nInformation\n\nRequest鉴权信息检索请求\

In [2]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.document_loaders import UnstructuredMarkdownLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from tqdm import tqdm
import os

# 对文本进行分块
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500, chunk_overlap=150)
split_docs = text_splitter.split_documents(docs)

# 加载开源词向量模型
embeddings = HuggingFaceEmbeddings(model_name="./embedding_model")

# 构建向量数据库
# 定义持久化路径
persist_directory = './data_base/vector_db/chroma'
# 加载数据库
vectordb = Chroma.from_documents(
    documents=split_docs,
    embedding=embeddings,
    persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
)
# 将加载的向量数据库持久化到磁盘上
vectordb.persist()

NameError: name 'docs' is not defined

In [17]:
from langchain.llms.base import LLM
from typing import Any, List, Optional
from langchain.callbacks.manager import CallbackManagerForLLMRun
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, LlamaTokenizerFast
import torch

class Qwen2_LLM(LLM):
    # 基于本地 Qwen2 自定义 LLM 类
    tokenizer: AutoTokenizer = None
    model: AutoModelForCausalLM = None
        
    def __init__(self, mode_name_or_path :str):

        super().__init__()
        print("正在从本地加载模型...")
        self.tokenizer = AutoTokenizer.from_pretrained(mode_name_or_path, use_fast=False)
        self.model = AutoModelForCausalLM.from_pretrained(mode_name_or_path, torch_dtype=torch.bfloat16, device_map="auto")
        self.model.generation_config = GenerationConfig.from_pretrained(mode_name_or_path)
        print("完成本地模型的加载")
        
    def _call(self, prompt : str, stop: Optional[List[str]] = None,
                run_manager: Optional[CallbackManagerForLLMRun] = None,
                **kwargs: Any):

        messages = [{"role": "user", "content": prompt }]
        input_ids = self.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
        model_inputs = self.tokenizer([input_ids], return_tensors="pt").to('cuda')
        generated_ids = self.model.generate(model_inputs.input_ids,max_new_tokens=512)
        generated_ids = [
            output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
        ]
        response = self.tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        
        return response
    @property
    def _llm_type(self) -> str:
        return "Qwen2_LLM"

In [19]:
# from LLM import Qwen2_LLM
llm = Qwen2_LLM(mode_name_or_path = "/root/autodl-tmp/langchainqwen14b/model_local/qwen/Qwen1.5-7B-Chat")
llm("你是谁")

正在从本地加载模型...


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|██████████| 4/4 [00:04<00:00,  1.06s/it]


完成本地模型的加载


/root/miniconda3/envs/langcha/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'我是阿里云推出的一种超大规模语言模型，我叫通义千问。'

In [20]:
from langchain.prompts import PromptTemplate

# 我们所构造的 Prompt 模板
template = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答案。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
有用的回答:"""

# 调用 LangChain 的方法来实例化一个 Template 对象，该对象包含了 context 和 question 两个变量，在实际调用时，这两个变量会被检索到的文档片段和用户提问填充
QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],template=template)

In [21]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})


In [23]:
question = "N7会话的ResourceURI由哪个网元在哪个消息中生成"
result = qa_chain({"query": question})
print("检索问答链回答 question 的结果：")
print(result["result"])

检索问答链回答 question 的结果：
N7会话的ResourceURI通常在UE（用户设备）会话的通信中生成。这通常发生在UE和网络服务器之间交互的消息中。


In [33]:
import jsonlines
from typing import Iterable
from tqdm import tqdm

def read_jsonl(path):
    content = []
    with jsonlines.open(path, "r") as json_file:
        for obj in json_file.iter(type=dict, skip_invalid=True):
            content.append(obj)
    return content


def save_answers(
    queries: Iterable, results: Iterable, path: str = "data/answers.jsonl"
):
    answers = []
    for query, result in zip(queries, results):
        answers.append(
            {"id": query["id"], "query": query["query"], "answer": result}
        )

    # use jsonlines to save the answers
    def write_jsonl(path, content):
        with jsonlines.open(path, "w") as json_file:
            json_file.write_all(content)

    # 保存答案到 data/answers.jsonl
    write_jsonl(path, answers)


queries = read_jsonl("./ops/question.jsonl")
print(queries)



[{'id': 1, 'query': 'PCF与NRF对接时，一般需要配置哪些数据？'}, {'id': 2, 'query': 'ZXUN RCP部署成功后，各个虚机个数都是最少个数，是否可以一次性扩容完成？有哪些注意事项？'}, {'id': 3, 'query': '如何排查PCF侧建立专载失败，发起Rx-ASR释放问题？'}, {'id': 4, 'query': 'Npcf_SMPolicyControl服务包含哪些操作？'}, {'id': 5, 'query': '什么是裸金属'}, {'id': 6, 'query': 'RCP策略决策包括哪些策略类型？'}, {'id': 7, 'query': 'Director中的VDC是什么概念？'}, {'id': 8, 'query': 'RCP在IaaS架构下有哪些GSU虚机？'}, {'id': 9, 'query': 'RCP如何将VoNR呼叫的Rx会话绑定到对应的N7会话？'}, {'id': 10, 'query': 'Director部署过程中有个网卡固化操作，那是在什么场景下做？'}, {'id': 11, 'query': '发布虚机时最多可以为虚机分配几块网卡？'}, {'id': 12, 'query': '虚拟资源包括哪些种类'}, {'id': 13, 'query': '物理机通用场景纳管1500台主机需要哪种部署规模、几个节点？'}, {'id': 14, 'query': 'EPS回落流程中，PCF向SBC上报几次用户位置吗，分别使用什么事件？'}, {'id': 15, 'query': '发布虚机前有哪些准备工作？'}, {'id': 16, 'query': '发布镜像时，镜像源支持哪几种方式？'}, {'id': 17, 'query': 'Director告警南向设置是在哪个界面上'}, {'id': 18, 'query': 'RCP包含哪些数据存储类服务？'}, {'id': 19, 'query': 'RCP对AF请求的授权响应方式有哪几种？'}, {'id': 20, 'query': 'Director接收南向告警的snmp端口号是多少'}, {'id': 21, 'query': 'RCP怎样实现语音会话隔离？'}, {'id': 22, 'query

In [34]:
# 生成答案
print("Start generating answers...")

results = []
for query in tqdm(queries, total=len(queries)):
# for query in queries:
    # result = await generation_with_knowledge_retrieval(
    #     query["query"], retriever, llm
    # )
    result = qa_chain({"query": query['query']})
    results.append(result["result"])

# 处理结果
save_answers(queries, results, "./result/submit_result.jsonl")

Start generating answers...


100%|██████████| 103/103 [05:06<00:00,  2.98s/it]
